In [1]:
import pandas as pd
from scipy.stats import ttest_ind, chi2_contingency
import sys
import os


In [2]:
# Load dataset
sys.path.append(os.path.abspath("../data"))
data = pd.read_csv(r"C:\Users\user\Projects\Insurance-Data-Analysis-for-Alpha_Care_Solutions\data\cleaned_insurance_data.csv")



C:\Users\user\AppData\Local\Temp\ipykernel_3944\1789327041.py:3: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r"C:\Users\user\Projects\Insurance-Data-Analysis-for-Alpha_Care_Solutions\data\cleaned_insurance_data.csv")


In [3]:
# Metrics
metric = 'TotalClaims'

# Data Segmentation
group_a = data[data['Province'] == 'Gauteng']
group_b = data[data['Province'] != 'Gauteng']

In [4]:
# Statistical Testing
# Numerical: t-test
stat, p_value_ttest = ttest_ind(group_a[metric], group_b[metric])
print(f"T-test p-value for {metric}: {p_value_ttest}")

T-test p-value for TotalClaims: 0.0009568315025529999


In [5]:
# Categorical: chi-squared
contingency_table = pd.crosstab(data['Gender'], data['Province'])
chi2_stat, p_value_chi2, dof, expected = chi2_contingency(contingency_table)
print(f"Chi-squared p-value for Gender vs Province: {p_value_chi2}")

Chi-squared p-value for Gender vs Province: 0.0


In [6]:
# Analyze and Report
if p_value_ttest < 0.05:
    print(f"Reject the null hypothesis: Significant difference in {metric} across provinces.")
else:
    print(f"Fail to reject the null hypothesis: No significant difference in {metric} across provinces.")

if p_value_chi2 < 0.05:
    print("Reject the null hypothesis: Significant association between Gender and Province.")
else:
    print("Fail to reject the null hypothesis: No significant association between Gender and Province.")


Reject the null hypothesis: Significant difference in TotalClaims across provinces.
Reject the null hypothesis: Significant association between Gender and Province.


In [7]:
numerical_columns = ['TotalPremium', 'TotalClaims', 'SumInsured', 'CalculatedPremiumPerTerm']

# Categorical distributions
categorical_columns = ['CoverCategory', 'CoverType', 'VehicleType', 'Province']


In [8]:
from sklearn.impute import SimpleImputer

# Numerical imputation
numerical_imputer = SimpleImputer(strategy='mean')
data[numerical_columns] = numerical_imputer.fit_transform(data[numerical_columns])

# Categorical imputation
categorical_imputer = SimpleImputer(strategy='most_frequent')
data[categorical_columns] = categorical_imputer.fit_transform(data[categorical_columns])


In [9]:
data['ProfitMargin'] = data['TotalPremium'] - data['TotalClaims']


In [10]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# One-hot encoding for categorical variables
data = pd.get_dummies(data, columns=['CoverType', 'Province'], drop_first=True)

# Label encoding for ordinal variables
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])


In [11]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['TotalClaims', 'TotalPremium'])  # Features
y = data['TotalPremium']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
datetime_columns = data.select_dtypes(include=['datetime64[ns]', 'object']).columns
print("Potential datetime columns:", datetime_columns)


Potential datetime columns: Index(['TransactionMonth', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Country', 'MainCrestaZone',
       'SubCrestaZone', 'ItemType', 'VehicleType', 'make', 'Model', 'bodytype',
       'VehicleIntroDate', 'AlarmImmobiliser', 'TrackingDevice',
       'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt',
       'Converted', 'CrossBorder', 'TermFrequency', 'ExcessSelected',
       'CoverCategory', 'CoverGroup', 'Section', 'Product', 'StatutoryClass',
       'StatutoryRiskType'],
      dtype='object')


In [15]:
# Convert 'TransactionMonth' to datetime
data['TransactionMonth'] = pd.to_datetime(data['TransactionMonth'], errors='coerce')

# Extract features from 'TransactionMonth'
data['Year'] = data['TransactionMonth'].dt.year
data['Month'] = data['TransactionMonth'].dt.month
data['Day'] = data['TransactionMonth'].dt.day
data['Weekday'] = data['TransactionMonth'].dt.weekday
data['IsWeekend'] = data['Weekday'].apply(lambda x: 1 if x >= 5 else 0)

# Drop the original 'TransactionMonth' column if not needed
data.drop(columns=['TransactionMonth'], inplace=True)


In [16]:
categorical_columns = [
    'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType',
    'MaritalStatus', 'Country', 'MainCrestaZone', 'SubCrestaZone', 'ItemType',
    'VehicleType', 'make', 'Model', 'bodytype', 'VehicleIntroDate',
    'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 'NewVehicle',
    'WrittenOff', 'Rebuilt', 'Converted', 'CrossBorder', 'TermFrequency',
    'ExcessSelected', 'CoverCategory', 'CoverGroup', 'Section', 'Product',
    'StatutoryClass', 'StatutoryRiskType'
]


In [17]:
from sklearn.preprocessing import LabelEncoder

# Apply label encoding to categorical columns
label_encoder = LabelEncoder()
for col in categorical_columns:
    data[col] = label_encoder.fit_transform(data[col].astype(str))


In [18]:
# Apply one-hot encoding
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)


In [19]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)


ValueError: could not convert string to float: '2015-04-01 00:00:00'

In [20]:
from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)


ValueError: could not convert string to float: '2015-04-01 00:00:00'

In [21]:
from sklearn.ensemble import RandomForestRegressor

forest_model = RandomForestRegressor(random_state=42)
forest_model.fit(X_train, y_train)


ValueError: could not convert string to float: '2015-04-01 00:00:00'

In [22]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(random_state=42)
xgb_model.fit(X_train, y_train)


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:TransactionMonth: object, Citizenship: object, LegalType: object, Title: object, Language: object, Bank: object, AccountType: object, MaritalStatus: object, Country: object, MainCrestaZone: object, SubCrestaZone: object, ItemType: object, VehicleType: object, make: object, Model: object, bodytype: object, VehicleIntroDate: object, AlarmImmobiliser: object, TrackingDevice: object, CapitalOutstanding: object, NewVehicle: object, WrittenOff: object, Rebuilt: object, Converted: object, CrossBorder: object, TermFrequency: object, ExcessSelected: object, CoverCategory: object, CoverGroup: object, Section: object, Product: object, StatutoryClass: object, StatutoryRiskType: object